<h1>Benchmark of some popular tools for Nucleic acids secondary structure prediction based on algorithmic approach</h1>

- RNAfold

- IPknot



<h3>Prerequisites</h3>

In [17]:
import pandas as pd

import time
from selenium import webdriver
from selenium.webdriver.common.by import By

from bs4 import BeautifulSoup

In [2]:
def find_basepairs(s):
    stack = []
    result = []

    for index, char in enumerate(s):
        if char == '(':
            stack.append(index)
        elif char == ')':
            # Если нашли закрывающую скобку, сопоставляем с последней открывающей
            if stack:
                open_index = stack.pop()
                result.append((open_index, index))

    return sorted(result)

def find_knotpairs(s):
    stack = []
    #result = {}
    result = []

    for index, char in enumerate(s):
        if char == '[':
            # Запоминаем индекс открывающей скобки
            stack.append(index)
        elif char == ']':
            # Если нашли закрывающую скобку, сопоставляем с последней открывающей
            if stack:
                open_index = stack.pop()
                #result[open_index] = index
                result.append((open_index, index))

    return sorted(result)

def find_otherpairs(s):
    stack = []
    #result = {}
    result = []

    for index, char in enumerate(s):
        if char == '{':
            # Запоминаем индекс открывающей скобки
            stack.append(index)
        elif char == '}':
            # Если нашли закрывающую скобку, сопоставляем с последней открывающей
            if stack:
                open_index = stack.pop()
                #result[open_index] = index
                result.append((open_index, index))

    return sorted(result)

def find_pairs(s):
    basepairs = find_basepairs(s)
    knotpairs = find_knotpairs(s)
    otherpairs = find_otherpairs(s)
    pairs = basepairs + knotpairs + otherpairs
    return sorted(pairs)



# def possible_pairs(s):

#     l = len(s)
#     pairs = []
#     for i in range(l):
#         for j in range(i+1,l):
#             pair = (i, j)
#             pairs.append(pair)

#     return pairs

#faster benchmark which do not calculate accuracy and TN
def Full_Benchmark_2Dv2(pred, ref, v=True):
    
    #Taminoto accuracy, adequate in case of g-quadruplexes predicted as '+' but not '[' and ']'
    tcounter = 0
    for i in range(len(ref)):
        if ref[i] == pred[i]:
            tcounter += 1

        elif (ref[i] != '.') and pred[i] == '+':
            tcounter += 1
    taminoto = tcounter/len(ref)


    #Binary classification metrics for all pairs
    pred_pairs = find_pairs(pred)
    ref_pairs = find_pairs(ref)
    # poss_pairs = possible_pairs(ref)
    
    # ref_notpairs = [x for x in poss_pairs if x not in ref_pairs] 
    # pred_notpairs = [x for x in poss_pairs if x not in pred_pairs]
    

    TP = len(set(pred_pairs).intersection(ref_pairs))
    FP = len(pred_pairs)-TP
    # TN = len(set(pred_notpairs).intersection(ref_notpairs)) 
    # FN = len(pred_notpairs)-TN
    FN = len(ref_pairs) -TP

    if TP == 0:
        TP = 0.0001

    if FP == 0:
        FP = 0.0001

    # if TN == 0:
    #     TN = 0.0001

    if FN == 0:
        FN = 0.0001
    
    # print(TP, TN, FP, FN, TP+FP+TN+FN)
    # print(len(pred_pairs), len(pred_notpairs), len(ref_pairs), len(ref_notpairs), len(poss_pairs))

    # Accuracy = (TP+TN)/(TP+TN+FP+FN)
    Precision = TP/(TP+FP)
    Recall = TP/(TP+FN)
    F1_score = (2*Precision*Recall)/(Precision+Recall)

    
    if v:
        print('Taminoto accuracy = ', round(taminoto, 3))
        # print('Accuracy =', round(Accuracy, 3))
        print('Precision =', round(Precision, 3))
        print('Recall =', round(Recall, 3))
        print('F1-score =', round(F1_score, 3))

    else:
        return taminoto, Precision, Recall, F1_score

#Taminoto accuracy, adequate in case of g-quadruplexes predicted as '+' but not '[' and ']'
def taminoto(pred, ref):
    tcounter = 0
    for i in range(len(ref)):
        if ref[i] == pred[i]:
            tcounter += 1

        elif (ref[i] != '.') and pred[i] == '+':
            tcounter += 1  

    return tcounter/len(ref)


def taminoto_accurate_percent(preds, refs, cutoff=0.85, v=True):
    
    acc_list = []
    acc = 0
    acc_counter = 0
    for i in range(len(preds)):
        acc = taminoto(preds[i], refs[i])
        if acc >= cutoff:
            acc_counter += 1
        acc_list.append(acc)

    if v:
        print(f'% of Accurate structures (taminoto acc. >= {cutoff}) =', round(100*acc_counter/len(preds), 2))

    else:
        return acc_counter/len(preds), acc_list

In [3]:
aptabase = pd.read_csv('data.csv')
def aptamers_to_fasta(db, filename):

    with open(filename, 'w') as fasta:

        ids = list(db['PDB'])
        seqs = list(db['Sequence'])

        for i in range(len(ids)):
            print('>'+ids[i], file=fasta)
            print(seqs[i], file=fasta)

aptamers_to_fasta(aptabase, 'data.fasta')

In [4]:
!RNAfold -g -v data.fasta --outfile=rnafold_predictions.fasta

Processing 1. input file "data.fasta"


In [5]:
!rm *.ps

In [6]:
def fastaparser(filename):

    with open(filename, 'r') as fasta:
        lines = fasta.readlines()
        preds = []
        for i in range(2, len(lines), 3):
            pred = lines[i].split()
            preds.append(pred[0])

    return preds

rnafold_preds = fastaparser('rnafold_predictions.fasta')
len(rnafold_preds)

354

In [7]:
aptabase['RNAfold_2D'] = pd.Series(rnafold_preds)

In [8]:
aptabase

,PDB,Strand,Sequence,2D,NAtype,RNAfold_2D
0,148D,strand_A,GGTTGGTGTGGTTGG,...(......)....,DNA,...............
1,1AW4,strand_A,ACCTGGGGGAGTATTGCGGAGGAAGGT,((((......((...))......)))),DNA,((((......((...))......))))
2,1DB6,strand_A,CGACCAACGTGTCGCCTGGTCG,((((((..(......))))))),DNA,((((((..........))))))
3,2ARG,strand_A,TGACCAGGGCAAACGGTAGGTGAGTGGTCA,((((((...[..((]....))...)))))),DNA,((((((......((.....))...))))))
4,2L5K,strand_A,GCAGTTGATCCTTTGGATACCCTG,.(((...((((...))))...))),DNA,.(((...((((...))))...)))
...,...,...,...,...,...,...
349,6P2H,strand_A,GGGUGUAAUCUCCAAAAUAUGGUUGGGGAGCCUCCACCAGUGAACC...,((((((..(((((((.....[[)))))))......).((((((.]]...,RNA,(((((...(((((((.......)))))))........((((((......
350,5FKE,strand_A,GGCUUAUCAAGAGAGGGUGAGCGACUGGCGCGAAGAGCCCCGGCAA...,((((((((....(.((((..(((.[.[[))).....)))))(((.....,RNA,((((((((......((((..(((.....))).....)))).((((....
351,2TOB,strand_A,ACUUGGUUUAGGUAAUGAGU,((((..(......)..)))),RNA,(((((...))))).......
352,1U1Y,strand_R,CCGGGGGAUCACCACGG,(((..((....)).))),RNA,(((((......)).)))


In [ ]:
#aptabase.to_csv('rnafold.csv')

In [ ]:
#RNA+DNA
rnafold_pred = ''.join(list(aptabase['RNAfold_2D']))
ref2d = ''.join(list(aptabase['2D']))
Full_Benchmark_2Dv2(rnafold_pred, ref2d)

Taminoto accuracy =  0.775
Precision = 0.719
Recall = 0.714
F1-score = 0.716


In [15]:
#RNA
aptabase_rna = aptabase[aptabase['NAtype'] == 'RNA']
rnafold_pred_rna = ''.join(list(aptabase_rna['RNAfold_2D']))
ref2d_rna = ''.join(list(aptabase_rna['2D']))
Full_Benchmark_2Dv2(rnafold_pred_rna, ref2d_rna)

Taminoto accuracy =  0.782
Precision = 0.731
Recall = 0.719
F1-score = 0.725


In [ ]:
#DNA
aptabase_dna = aptabase[aptabase['NAtype'] == 'DNA']
rnafold_pred_dna = ''.join(list(aptabase_dna['RNAfold_2D']))
ref2d_dna = ''.join(list(aptabase_dna['2D']))
Full_Benchmark_2Dv2(rnafold_pred_dna, ref2d_dna)

Taminoto accuracy =  0.695
Precision = 0.567
Recall = 0.654
F1-score = 0.607


In [51]:
def ipknoter_v1(seq):

    driver = webdriver.Chrome()


    s = seq.replace('T', 'U')

    # Open web-server
    driver.get('http://ws.sato-lab.org/rtips/ipknot++/')  # Replace with your target URL
    
    # type sequence
    input_seq = driver.find_element(By.NAME, "seq")
    input_seq.send_keys(s)

    #settings
    # select_level = driver.find_element(By.NAME, 'level')
    # select_level.selectByValue('3')

    # select_engine = driver.find_element(By.NAME, 'model')
    # select_engine.select_by_value('NUPACK')

    select_level3 = driver.find_element(By.XPATH, '/html/body/div/form/ul/li[1]/select/option[3]')
    select_level3.click()

    select_nupack = driver.find_element(By.XPATH, '/html/body/div/form/ul/li[2]/select/option[6]')
    select_nupack.click()

    #click pred button
    predbutton = driver.find_element(By.XPATH, "/html/body/div/form/p[3]/input[1]")
    predbutton.click()

    # Wait for the page to load 
    time.sleep(5)

    
    # Get the resulting HTML
    result_html = driver.page_source
    driver.quit()

   
    
    #we need text from xpath /html/body/div/div[2]/pre[3]
    # Создаём объект BeautifulSoup
    soup = BeautifulSoup(result_html, 'html.parser')

    # CSS-селектор, соответствующий вашему XPath
    css_selector = 'body > div > div:nth-of-type(2) > pre:nth-of-type(3)'

    # Находим элемент
    element = soup.select_one(css_selector)
    text = element.get_text()
    return text.split()[1]

In [53]:
#ipknot most accurate (and computational-costly) parameteres - level3, nupack model
#it has limitation - length of one sequence no more than 80 nt
aptabase_ipk = aptabase[aptabase['Sequence'].apply(lambda x: len(x)) <= 80]

print(len(aptabase_ipk))
seqs = list(aptabase_ipk['Sequence'])
ipknotv1_preds = []
for seq in seqs:
    ipknotv1_preds.append(ipknoter_v1(seq))
    print(len(ipknotv1_preds), '-', end=' ')

237
1 - 2 - 3 - 4 - 5 - 6 - 7 - 8 - 9 - 10 - 11 - 12 - 13 - 14 - 15 - 16 - 17 - 18 - 19 - 20 - 21 - 22 - 23 - 24 - 25 - 26 - 27 - 28 - 29 - 30 - 31 - 32 - 33 - 34 - 35 - 36 - 37 - 38 - 39 - 40 - 41 - 42 - 43 - 44 - 45 - 46 - 47 - 48 - 49 - 50 - 51 - 52 - 53 - 54 - 55 - 56 - 57 - 58 - 59 - 60 - 61 - 62 - 63 - 64 - 65 - 66 - 67 - 68 - 69 - 70 - 71 - 72 - 73 - 74 - 75 - 76 - 77 - 78 - 79 - 80 - 81 - 82 - 83 - 84 - 85 - 86 - 87 - 88 - 89 - 90 - 91 - 92 - 93 - 94 - 95 - 96 - 97 - 98 - 99 - 100 - 101 - 102 - 103 - 104 - 105 - 106 - 107 - 108 - 109 - 110 - 111 - 112 - 113 - 114 - 115 - 116 - 117 - 118 - 119 - 120 - 121 - 122 - 123 - 124 - 125 - 126 - 127 - 128 - 129 - 130 - 131 - 132 - 133 - 134 - 135 - 136 - 137 - 138 - 139 - 140 - 141 - 142 - 143 - 144 - 145 - 146 - 147 - 148 - 149 - 150 - 151 - 152 - 153 - 154 - 155 - 156 - 157 - 158 - 159 - 160 - 161 - 162 - 163 - 164 - 165 - 166 - 167 - 168 - 169 - 170 - 171 - 172 - 173 - 174 - 175 - 176 - 177 - 178 - 179 - 180 - 181 - 182 - 183 - 184 - 

In [ ]:
aptabase_ipk['ipknotv1'] = pd.Series(ipknotv1_preds)
aptabase_ipk.to_csv('ipknotv1.csv')

/tmp/ipykernel_226340/649737437.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  aptabase_ipk['ipknotv1'] = pd.Series(ipknotv1_preds)


In [67]:
aptabase_ipk = pd.read_csv('ipknotv1.csv').drop('Unnamed: 0', axis=1)
aptabase_ipk

,PDB,Strand,Sequence,2D,NAtype,RNAfold_2D,ipknotv1
0,148D,strand_A,GGTTGGTGTGGTTGG,...(......)....,DNA,...............,...............
1,1AW4,strand_A,ACCTGGGGGAGTATTGCGGAGGAAGGT,((((......((...))......)))),DNA,((((......((...))......)))),((((...................))))
2,1DB6,strand_A,CGACCAACGTGTCGCCTGGTCG,((((((..(......))))))),DNA,((((((..........)))))),((((((..........))))))
3,2ARG,strand_A,TGACCAGGGCAAACGGTAGGTGAGTGGTCA,((((((...[..((]....))...)))))),DNA,((((((......((.....))...)))))),((((((..((..[[.))..]]...))))))
4,2L5K,strand_A,GCAGTTGATCCTTTGGATACCCTG,.(((...((((...))))...))),DNA,.(((...((((...))))...))),.(((...((((...))))...)))
...,...,...,...,...,...,...,...
232,9BUN,strand_A,GGACUCGGAAACGUGAAGGAGAGGCGCAAGGUUAACCGCCUCAGUCCA,(((((((....))([...[)((((((]........]))))))))))).,RNA,(((((((....)).......((((((..........))))))))))).,NaN
233,6P2H,strand_A,GGGUGUAAUCUCCAAAAUAUGGUUGGGGAGCCUCCACCAGUGAACC...,((((((..(((((((.....[[)))))))......).((((((.]]...,RNA,(((((...(((((((.......)))))))........((((((......,NaN
234,2TOB,strand_A,ACUUGGUUUAGGUAAUGAGU,((((..(......)..)))),RNA,(((((...))))).......,NaN
235,1U1Y,strand_R,CCGGGGGAUCACCACGG,(((..((....)).))),RNA,(((((......)).))),NaN


In [74]:
#RNA+DNA
aptabase_ipk = aptabase_ipk.dropna()
ipknotv1_pred = ''.join(ipknotv1_preds)
ref2d = ''.join(list(aptabase_ipk['2D']))
#
Full_Benchmark_2Dv2(ipknotv1_pred, ref2d)

Taminoto accuracy =  0.674
Precision = 0.378
Recall = 0.581
F1-score = 0.458


In [106]:
#other settings: all default 
def ipknoter_v2(seq):

    driver = webdriver.Chrome()


    s = seq.replace('T', 'U')

    # Open web-server
    driver.get('http://ws.sato-lab.org/rtips/ipknot++/')  # Replace with your target URL
    
    # type sequence
    input_seq = driver.find_element(By.NAME, "seq")
    input_seq.send_keys(s)

    #settings
    # select_level = driver.find_element(By.NAME, 'level')
    # select_level.selectByValue('3')

    # select_engine = driver.find_element(By.NAME, 'model')
    # select_engine.select_by_value('NUPACK')

    #select_level3 = driver.find_element(By.XPATH, '/html/body/div/form/ul/li[1]/select/option[2]')
    #select_level3.click()

    #select_nupack = driver.find_element(By.XPATH, '/html/body/div/form/ul/li[2]/select/option[6]')
    #select_nupack.click()

    #click pred button
    predbutton = driver.find_element(By.XPATH, "/html/body/div/form/p[3]/input[1]")
    predbutton.click()

    # Wait for the page to load 
    time.sleep(5)

    
    # Get the resulting HTML
    result_html = driver.page_source
    driver.quit()

   
    
    #we need text from xpath /html/body/div/div[2]/pre[3]
    # Создаём объект BeautifulSoup
    soup = BeautifulSoup(result_html, 'html.parser')

    # CSS-селектор, соответствующий вашему XPath
    css_selector = 'html body div#container div#vienna'

    # Находим элемент
    element = soup.select_one(css_selector)
    text = element.get_text()
    

    ans = ''
    for l in text:
        if l in ('(', ')', '[', ']', '.'):
            ans += l
    return ans
    #return text.split()

In [ ]:
seqs = list(aptabase['Sequence'])
ipknotv2_preds = []
for seq in seqs:
    ipknotv2_preds.append(ipknoter_v2(seq))
    print(len(ipknotv2_preds), '-', end='', sep='')

In [116]:
len(ipknotv2_preds)

188

In [ ]:
for seq in seqs[188:]:
    ipknotv2_preds.append(ipknoter_v2(seq))
    print(len(ipknotv2_preds), '-', end='', sep='')

In [118]:
aptabase['ipknot'] = ipknotv2_preds

In [120]:
#RNA+DNA

ipknotv2_pred = ''.join(ipknotv2_preds)
ref2d = ''.join(list(aptabase['2D']))
#
Full_Benchmark_2Dv2(ipknotv2_pred, ref2d)

Taminoto accuracy =  0.726
Precision = 0.776
Recall = 0.72
F1-score = 0.747


In [122]:
#RNA
aptabase_rna = aptabase[aptabase['NAtype'] == 'RNA']
ipknotv2_pred_rna = ''.join(list(aptabase_rna['ipknot']))
ref2d_rna = ''.join(list(aptabase_rna['2D']))
Full_Benchmark_2Dv2(ipknotv2_pred_rna, ref2d_rna)

Taminoto accuracy =  0.728
Precision = 0.788
Recall = 0.725
F1-score = 0.755


In [123]:
#DNA
aptabase_dna = aptabase[aptabase['NAtype'] == 'DNA']
ipknotv2_pred_dna = ''.join(list(aptabase_dna['ipknot']))
ref2d_dna = ''.join(list(aptabase_dna['2D']))
Full_Benchmark_2Dv2(ipknotv2_pred_dna, ref2d_dna)

Taminoto accuracy =  0.7
Precision = 0.631
Recall = 0.639
F1-score = 0.635


In [125]:
aptabase.to_csv('rnafold_ipknot.csv')